In [51]:
import os
import ast
import json

In [73]:
def IsTestFile(filename: str) -> bool:
    filename = filename.lower()
    if filename.endswith(".py") and "test" in filename:
        return True
    return False


def WriteToJson(filename:str, data:list):
    with open(filename,"w") as f:
        json.dump(data, f, indent=4)

In [74]:
def ExtractTestData(test_dir:str, code_type:str)->list:
    data = []
    for root, _, files in os.walk(test_dir):
        for file in files:
            if IsTestFile(file):
                file_path = os.path.join(root, file)

                with open(file_path, "r", encoding="utf-8") as f:
                    try:
                        tree = ast.parse(f.read())
                    except:
                        continue
                
                for node in ast.walk(tree):
                    if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                        start_line = node.lineno
                        end_line = node.end_lineno
                        num_lines = end_line-start_line
                        if num_lines >= 5 and num_lines <= 35:
                            with open(file_path, 'r', encoding='utf-8') as f:
                                lines = f.readlines()
                                code_block = ''.join(lines[start_line - 1:end_line])
                                loc = "\\".join(file_path.split("\\")[11:])  # hardcode: index
                                dic = {"code":code_block, "type":code_type, "loc":loc}
                                data.append(dic)

    return data

In [77]:
dj_test_dir = "C:\\Users\\Ting\\Desktop\\TingTing\\UWaterloo\\courses\\Fall 2024\\CS846\\project\\test\\django\\tests"
code_type = "django"
data = ExtractTestData(dj_test_dir, code_type)
WriteToJson("dj_test.json",data)

In [78]:
fl_test_dir = "C:\\Users\\Ting\\Desktop\\TingTing\\UWaterloo\\courses\\Fall 2024\\CS846\\project\\test\\flask\\tests"
code_type = "flask"
data = ExtractTestData(fl_test_dir, code_type)
WriteToJson("fl_test.json",data)